<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/DGRNet%20STEP3-%20Hourly%20Data%20-%20MLP%20Grid%20Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from keras import backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping
import keras.backend as K
from keras.callbacks import Callback
import plotly
import plotly.express as px # for data visualization
import seaborn as sns

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
!pip install scikit-learn==0.24.2



generator_multiply = 1 #each input record will generate 100 random vectors from the latent space, given the mu and sigma generated by the encoder

train_data = np.load(r'/content/drive/MyDrive/PHD/2024/multivariate_long_sequences-TRAIN_hourly.npy') #------for Hourly data
index = 500
#We missed i=500 from processing the iosw. So here we are dropping row with index =500
train_data= np.delete(train_data, index, axis=0)
window_label = np.load(r'/content/drive/MyDrive/PHD/2024/multivariate_long_sequences_WINDOW-TRAIN_hourly.npy')
n_seq = train_data.shape[0]
window_size = train_data.shape[1]
n_features = train_data.shape[2]

##---------------------------IGNORE THIS IF NOT GENERATING FRESH VAE DATASET-------------------------------------------------

encoder = keras.models.load_model(r'/content/drive/MyDrive/PHD/2024/VAE_SIMULATION/vae-encoder-saved-hourly-latent10-dim128-latest.model')
decoder = keras.models.load_model(r'/content/drive/MyDrive/PHD/2024/VAE_SIMULATION/vae-decoder-saved-hourly-latent10-dim128-latest.model')

X_train_encoded = encoder.predict(train_data)
mu, logvar, z = X_train_encoded
sigma = tf.exp(0.5 * logvar)
batch = tf.shape(mu)[0]  #number of recors / batchs
dim = tf.shape(mu)[1] #Ndimension of latent variable
store = list()
storetemp = list()

#For each batch, iterate, get the generator_multipy number of latent vectors with same window_size.
#For each z, concatenate z_mean, so it will become 100 dimensional vector

for i in range(0,batch):
  all_Z_i = tf.random.normal(shape=(generator_multiply,dim), mean = mu[i,:], stddev=sigma[i,:]) #all randorm vectors for this record i
  X_train_decoded = decoder.predict(all_Z_i)
  X_train_decoded = X_train_decoded.reshape((X_train_decoded.shape[0],window_size*n_features))
  a = np.arange(generator_multiply)
  a.fill(window_label[i])
  c=np.concatenate(((X_train_decoded,a[:,None])),axis=1)
  store.append(c)

results1=np.concatenate(store,axis=0)
np.save(r'/content/drive/MyDrive/PHD/2024/labelled_subsquence_data_hourly',results1)

##########---------------------------------------------------------------------------------------------------------------------------------------------------


results1=np.load(r'/content/drive/MyDrive/PHD/2024/labelled_subsquence_data_hourly.npy')
x=results1[:,:-1]
y=results1[:,window_size*n_features]


from sklearn.ensemble import IsolationForest
iso = IsolationForest(contamination=0.4)
yhat = iso.fit_predict(x)
# select all rows that are not outliers
mask = yhat != -1
x, y = x[mask, :], y[mask]


###############Scale the target -----------------------------------------------------
from sklearn.preprocessing import StandardScaler
transformer = StandardScaler()

y_transformed = transformer.fit_transform(y.reshape(-1,1)).flatten()

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y_transformed, test_size = 0.1, random_state = 42)

#----------------------------first try sckit learn to pick best model--------------
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
mlp = MLPRegressor(max_iter=100)
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,30,15,5)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3,scoring = 'r2',vurbose = 1)
clf.fit(x_train, y_train)



# All results
means = clf.cv_scores_['mean_test_score']

stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_scores_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

# summarize results
print('Best score: %s' % clf.best_score_)
print('Best parameters set:')

y_true, y_pred = DEAP_y_test , clf.predict(DEAP_x_test)

from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_true, y_pred))


# create model
model = KerasRegressor(model=create_model, verbose=0)
#reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
es = keras.callbacks.EarlyStopping(patience=20, verbose=1, min_delta=0.000001, monitor='loss', mode='auto', restore_best_weights=True)
n_epochs = 500
# define the grid search parameters
batch_size = [32, 50, 100]
epochs = [100, 500]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

#model.fit(x_train, y_train,epochs=5, batch_size=50, verbose=True)

#y_pred = model.predict(x_test)

#transform

#x_val = x_train[10001:13000]
#y_val = y_train_transformed[10001:13000]


history=model.fit( x_train,y_train,
                 epochs=n_epochs,
                 batch_size=32,
                   #validation_data=(x_val,y_val),
                   validation_split=0.1)
                 #callbacks=[es])

y_train_pred = model.predict(x_train)
y_train_pred = transformer.inverse_transform(y_train_pred)

y_pred = model.predict(x_test)
y_pred = transformer.inverse_transform(y_pred)

y_train = transformer.inverse_transform(y_train.reshape(-1,1)).flatten()
y_test = transformer.inverse_transform(y_test.reshape(-1,1)).flatten()



score_train= r2_score(y_train,y_train_pred)
print("r2 score is ==",score_train)

score= r2_score(y_test,y_pred)
print("r2 score is ==",score)


#plt.scatter(y_test,y_pred);
#plt.xlabel('Actual');
#plt.ylabel('Predicted');
plt.plot(y_train[0:100], color = 'red', label = 'Real data')
plt.plot(y_train_pred[0:100], color = 'blue', label = 'Predicted data')
plt.title('Prediction')
plt.legend()
plt.show()

#plt.scatter(y_test,y_pred);
#plt.xlabel('Actual');
#plt.ylabel('Predicted');
plt.plot(y_test[100:150], color = 'red', label = 'Real data')
plt.plot(y_pred[100:150], color = 'blue', label = 'Predicted data')
plt.title('Prediction')
plt.legend()
plt.show()




np.savetxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/preduber_2.csv',y_pred)
np.savetxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/realuber_2.csv',y_test)
print("MAE is==",mean_absolute_error(y_test,y_pred))


  Using cached scikit-learn-0.24.2.tar.gz (7.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


1/1 [==============================] - 0s 79ms/step


In [ ]:
from google.colab import drive
drive.mount('/content/drive')